# Open Notebook & Additional Resources

<a target="_blank" href="https://colab.research.google.com/github/Nicolepcx/ORM-self-improving-ai-agents-course/blob/main/hands_on/session_04_HANDS_ON_additional_histories.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://learning.oreilly.com/library/view/ai-agents-the/0642572247775/">
  <img src="https://img.shields.io/badge/AI%20Agents%20Book-Read%20on%20O'Reilly-d40101?style=flat" alt="AI Agents Book – Read on O'Reilly"/>
</a>





<font color="red" size="10">
<b>HANDS-ON TIME: 15 mins</b>
</font>

# About this Notebook

## What to do first (read this now)

During the live session, **you do NOT need to read everything below**.

**What you should do right now:**

1. **Run the notebook as is**
2. Look at the three example trajectories:
   - Preserving special tokens
   - Sub-agent delegation
   - History compaction
3. Inspect:
   - `messages_and_choices`
   - `additional_histories`
   - `metrics`
4. Observe how *one logical task* can contain **multiple conversations**

That’s it.  
No coding required during the session.

👉 **After the course**, come back and read the rest of this notebook for the deeper explanation of *why* this matters and *how* to use it in production systems.

---

## What this notebook is really about (read later)

This notebook introduces **additional histories**, a mechanism for storing *more than one conversation* inside a single training trajectory.

In real agent systems, the main user-facing conversation is often **not** where the work happens. Important behavior occurs in:

* sub-agent conversations
* tool-call traces
* reasoning tokens removed by chat templates
* long conversations that later get summarized

If you only train on the final coordinator transcript, you lose the information that caused the reward. That breaks credit assignment and slows learning.

Additional histories solve this by letting you attach those hidden or intermediate conversations directly to the trajectory.

---

## The three examples you just ran

### 1. Preserving special tokens

Some models or chat templates remove tokens like `<think>` from earlier turns.

This example shows how to:
* keep the original reasoning tokens
* store a cleaned version separately
* train on both

---

### 2. Sub-agent delegation

The coordinator delegates work to sub-agents.

Instead of losing those conversations, they are stored as additional histories so the model can learn:
* how to delegate
* how to evaluate sub-agent output
* how to combine results

---

### 3. History compaction

Long-running agents often summarize past conversations.

This example keeps:
* the compacted summary in the main history
* the original conversation in an additional history

This preserves training signal without increasing runtime context.

---

## Evaluation note

RULER does not support `additional_histories` yet.

That’s why evaluation in this notebook uses a simple reward function over the main trajectory only.  
Additional histories are currently meant for **training signal**, not evaluation.

---

## One takeaway

If you remember one thing:

**The final chat transcript is not the full behavior of an agent.  
Additional histories are how you train on what actually happened.**


# Timer

In [ ]:
SET_TIMER = False  # False, True, or minutes as a number

import requests, types
url = "https://raw.githubusercontent.com/Nicolepcx/ORM-self-improving-ai-agents-course/main/timer.py"

timer = types.ModuleType("timer")
exec(requests.get(url).text, timer.__dict__)

timer.start_exam_timer(enabled=SET_TIMER, minutes=15, warn_minutes=5)

In [7]:
# @title Installation
# Portions adapted from Unsloth Notebooks (https://github.com/unslothai/notebooks)
# Copyright (c) Unsloth contributors.
# License: GNU LGPL v3.0.
# Modifications by OpenPipe:
# See /licenses/LGPL-3.0.txt and /licenses/GPL-3.0.txt for full text.

%%capture
import os

if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install openpipe-art[backend]==0.4.11 tenacity "mcp>=1.11.0" "gql<4" aiohttp --prerelease allow --no-cache-dir
else:
    try:
        import numpy

        get_numpy = f"numpy=={numpy.__version__}"
    except:
        get_numpy = "numpy"
    try:
        import subprocess

        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_t4 = False
    get_vllm, get_triton = (
        ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    )
    !uv pip install --upgrade \
        openpipe-art[backend]==0.4.11 tenacity pillow==11.3.0 protobuf==5.29.5 {get_vllm} {get_numpy} --prerelease allow --no-cache-dir
    !uv pip install -qqq {get_triton}

# Imports

In [8]:
import art
from art.trajectories import Trajectory, History, Choice
from openai.types.chat import ChatCompletionMessage
from art.utils.logging import info, ok, step

## Why Additional Histories?

In a multi-agent system, if you only train on the main coordinator conversation, the model sees:
- ✅ The final output
- ❌ But NOT the sub-agent reasoning, actions, and intermediate steps

This breaks **credit assignment** - the model can't learn:
- How to delegate effectively
- How to verify sub-agent work
- How to compose results from multiple agents

By storing each sub-agent conversation as an **additional history**, you give the RL training process:
- The functions and actions each sub-agent used
- The intermediate signals and reasoning
- Clear attribution of what led to the final reward

---

# Hands-on

<font color="red" size="10">
<b>TODO: </b>
</font>
<br>
<font color="black" size="5">
<b>Create Trajectories with Additional Histories</b>
</font>


In this hands-on, you'll create three different trajectories (just run the notebook) demonstrating the use cases for additional histories:

1. **Preserving Special Tokens** - Keep reasoning tokens across multi-turn conversations
2. **Sub-Agent Delegation** - Train on both coordinator and sub-agent conversations
3. **History Compaction** - Preserve original conversation before summarization

You'll also see how to evaluate trajectories without RULER (since RULER doesn't support additional histories yet).

In [9]:
# Example 1: Preserving Special Tokens in Multi-Turn Conversations
# Some models remove special tokens like <think> from previous turns
# By using additional histories, we can preserve these tokens for training

from art.trajectories import Trajectory, History, Choice
from openai.types.chat import ChatCompletionMessage

trajectory_preserve_tokens = Trajectory(
    messages_and_choices=[
        # First turn with thinking preserved
        {"role": "user", "content": "What is 2+2?"},
        Choice(
            index=0,
            finish_reason="stop",
            message=ChatCompletionMessage(
                role="assistant",
                content="<think>I need to add 2 and 2</think>4",
            ),
        ),
    ],
    additional_histories=[
        History(
            messages_and_choices=[
                # The Qwen 3 chat template removes <think> tokens from previous turns
                # So we store the "cleaned" version in an additional history
                {"role": "user", "content": "What is 2+2?"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(role="assistant", content="4"),
                ),
                {"role": "user", "content": "What is 3+3?"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(
                        role="assistant",
                        content="<think>I need to add 3 and 3</think>6",
                    ),
                ),
            ]
        )
    ],
    reward=0.9,
    metrics={"preserved_reasoning_tokens": True},
)

print("Example 1: Preserving Special Tokens")
print("=" * 60)
print(f"Primary history messages: {len(trajectory_preserve_tokens.messages_and_choices)}")
print(f"Additional histories: {len(trajectory_preserve_tokens.additional_histories)}")
print(f"Reward: {trajectory_preserve_tokens.reward}")
print("\nPrimary history:")
for msg in trajectory_preserve_tokens.messages_and_choices:
    if isinstance(msg, dict):
        print(f"  {msg['role']}: {msg['content'][:80]}...")
    else:
        content = msg.message.content if msg.message.content else ""
        print(f"  Choice: {content[:80]}...")

Example 1: Preserving Special Tokens
Primary history messages: 2
Additional histories: 1
Reward: 0.9

Primary history:
  user: What is 2+2?...
  Choice: <think>I need to add 2 and 2</think>4...


## Example 2: Training Agents That Call Sub-Agents (Sub-Agent Delegation)

When an agent delegates work to sub-agents, each sub-agent conversation can be stored as an additional history. This allows the model to learn from both the main conversation flow and the sub-agent interactions.

In [10]:
# Main agent delegates work to specialized sub-agents

ANALYZE_ID = "call_analyze_1"
FIX_ID = "call_fix_1"

trajectory_sub_agents = Trajectory(
    # Main agent conversation
    messages_and_choices=[
        {"role": "user", "content": "Analyze this codebase and fix any bugs"},

        # Assistant tool call: analyze_code
        Choice(
            index=0,
            finish_reason="tool_calls",
            message=ChatCompletionMessage(
                role="assistant",
                content=None,
                tool_calls=[
                    {
                        "id": ANALYZE_ID,
                        "type": "function",
                        "function": {
                            "name": "analyze_code",
                            "arguments": '{"file": "main.py", "task": "Find potential bugs"}',
                        },
                    }
                ],
            ),
        ),

        # Tool result for analyze_code
        {
            "role": "tool",
            "tool_call_id": ANALYZE_ID,
            "content": (
                "Found 3 potential issues: null pointer on line 42, "
                "memory leak on line 87, uninitialized variable on line 120"
            ),
        },

        # Assistant tool call: fix_issues
        Choice(
            index=0,
            finish_reason="tool_calls",
            message=ChatCompletionMessage(
                role="assistant",
                content=None,
                tool_calls=[
                    {
                        "id": FIX_ID,
                        "type": "function",
                        "function": {
                            "name": "fix_issues",
                            "arguments": '{"issue": "null pointer on line 42", "fix": "Add null check"}',
                        },
                    }
                ],
            ),
        ),

        # Tool result for fix_issues
        {
            "role": "tool",
            "tool_call_id": FIX_ID,
            "content": "Fixed by adding null check: if (ptr != nullptr) { ... }",
        },

        # Final assistant answer
        Choice(
            index=0,
            finish_reason="stop",
            message=ChatCompletionMessage(
                role="assistant",
                content=(
                    "I've analyzed the codebase and fixed the critical null pointer issue on line 42. "
                    "The fix adds a null check before dereferencing the pointer."
                ),
            ),
        ),
    ],
    additional_histories=[
        # Sub-agent 1: Code analysis
        History(
            messages_and_choices=[
                {"role": "system", "content": "You are a code analysis expert. Analyze code for potential bugs."},
                {"role": "user", "content": "Find potential bugs in main.py"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(
                        role="assistant",
                        content=(
                            "Found 3 potential issues:\n"
                            "1. Null pointer on line 42\n"
                            "2. Memory leak on line 87\n"
                            "3. Uninitialized variable on line 120"
                        ),
                    ),
                ),
            ]
        ),
        # Sub-agent 2: Bug fixing
        History(
            messages_and_choices=[
                {"role": "system", "content": "You are a bug fixing expert. Fix code issues safely and correctly."},
                {"role": "user", "content": "Fix the null pointer issue on line 42"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(
                        role="assistant",
                        content=(
                            "Fixed by adding null check:\n"
                            "```cpp\n"
                            "if (ptr != nullptr) {\n"
                            "    // safe to use ptr\n"
                            "}\n"
                            "```"
                        ),
                    ),
                ),
            ]
        ),
    ],
    reward=0.85,
    metrics={
        "sub_agents_used": 2,
        "bugs_found": 3,
        "bugs_fixed": 1,
        "task_completed": True,
    },
)

print("Example 2: Sub-Agents")
print("=" * 60)
print(f"Primary history messages: {len(trajectory_sub_agents.messages_and_choices)}")
print(f"Additional histories: {len(trajectory_sub_agents.additional_histories)}")
print(f"Reward: {trajectory_sub_agents.reward}")
print(f"Metrics: {trajectory_sub_agents.metrics}")
print("\nSub-agents:")
for i, history in enumerate(trajectory_sub_agents.additional_histories, 1):
    print(f"\n  Sub-agent {i}:")
    for msg in history.messages_and_choices:
        if isinstance(msg, dict):
            role = msg.get('role', 'unknown')
            content = msg.get('content', '')[:100]
            print(f"    {role}: {content}...")
        else:
            content = msg.message.content if msg.message.content else ""
            print(f"    Choice: {content[:100]}...")

Example 2: Sub-Agents
Primary history messages: 6
Additional histories: 2
Reward: 0.85
Metrics: {'sub_agents_used': 2, 'bugs_found': 3, 'bugs_fixed': 1, 'task_completed': True}

Sub-agents:

  Sub-agent 1:
    system: You are a code analysis expert. Analyze code for potential bugs....
    user: Find potential bugs in main.py...
    Choice: Found 3 potential issues:
1. Null pointer on line 42
2. Memory leak on line 87
3. Uninitialized vari...

  Sub-agent 2:
    system: You are a bug fixing expert. Fix code issues safely and correctly....
    user: Fix the null pointer issue on line 42...
    Choice: Fixed by adding null check:
```cpp
if (ptr != nullptr) {
    // safe to use ptr
}
```...


# Example 3: History Compaction
## Long-running agent that compresses conversation history periodically


In [11]:

trajectory_compaction = Trajectory(
    # Current active conversation (after compaction)
    messages_and_choices=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": (
                "Compacted conversation history: The user asked about quantum entanglement, and I explained the "
                "basic concept of how particles can be correlated even when separated."
            ),
        },
        {"role": "user", "content": "Tell me more about the history of quantum entanglement"},
        Choice(
            index=0,
            finish_reason="stop",
            message=ChatCompletionMessage(
                role="assistant",
                content=(
                    "Quantum entanglement was first proposed by Einstein, Podolsky, and Rosen in 1935 in what's now "
                    "called the EPR paradox. They were concerned that quantum mechanics seemed to violate locality, "
                    "the idea that objects can only be influenced by their immediate surroundings. However, "
                    "experiments by John Bell in the 1960s and subsequent tests have confirmed that entanglement is "
                    "a real physical phenomenon. Today, entanglement is a key resource in quantum computing and "
                    "quantum cryptography."
                ),
            ),
        ),
        {"role": "user", "content": "How is it used in quantum computing?"},
        Choice(
            index=0,
            finish_reason="stop",
            message=ChatCompletionMessage(
                role="assistant",
                content=(
                    "In quantum computing, entanglement is used to create quantum gates that operate on multiple "
                    "qubits simultaneously. For example, the CNOT gate creates entanglement between two qubits. "
                    "This enables quantum algorithms to process information in ways that classical computers cannot. "
                    "Some algorithms show large speedups for specific tasks, such as factoring (Shor's algorithm) or "
                    "search (Grover's algorithm)."
                ),
            ),
        ),
    ],
    additional_histories=[
        History(
            messages_and_choices=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Explain quantum entanglement"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(
                        role="assistant",
                        content=(
                            "Quantum entanglement is a phenomenon where two or more particles become correlated so "
                            "that the quantum state of each particle cannot be described independently, even when the "
                            "particles are separated by large distances. Measuring one particle lets you predict "
                            "properties of the other, regardless of distance. This 'spooky action at a distance' "
                            "(as Einstein called it) has been experimentally verified and is a fundamental aspect of "
                            "quantum mechanics."
                        ),
                    ),
                ),
                {"role": "user", "content": "Can you give me a simple example?"},
                Choice(
                    index=0,
                    finish_reason="stop",
                    message=ChatCompletionMessage(
                        role="assistant",
                        content=(
                            "Imagine two coins that are perfectly anti-correlated. If one is heads, the other is "
                            "tails, every time. If you separate them far apart, observing one instantly tells you "
                            "what you'd observe for the other. This is only an analogy: real entanglement is not "
                            "about hidden predetermined coin states, but it helps build intuition."
                        ),
                    ),
                ),
            ]
        )
    ],
    reward=0.92,
    metrics={
        "history_compacted": True,
        "original_turns": 4,
        "compacted_turns": 3,
    },
)

print("Example 3: History Compaction")
print("=" * 60)
print(f"Primary history messages: {len(trajectory_compaction.messages_and_choices)}")
print(f"Additional histories: {len(trajectory_compaction.additional_histories)}")
print(f"Reward: {trajectory_compaction.reward}")
print(f"Metrics: {trajectory_compaction.metrics}")
print("\nOriginal conversation (before compaction):")
for msg in trajectory_compaction.additional_histories[0].messages_and_choices:
    if isinstance(msg, dict):
        role = msg.get('role', 'unknown')
        content = msg.get('content', '')[:100]
        print(f"  {role}: {content}...")
    else:
        content = msg.message.content if msg.message.content else ""
        print(f"  Choice: {content[:100]}...")

Example 3: History Compaction
Primary history messages: 6
Additional histories: 1
Reward: 0.92
Metrics: {'history_compacted': True, 'original_turns': 4, 'compacted_turns': 3}

Original conversation (before compaction):
  system: You are a helpful assistant....
  user: Explain quantum entanglement...
  Choice: Quantum entanglement is a phenomenon where two or more particles become correlated so that the quant...
  user: Can you give me a simple example?...
  Choice: Imagine two coins that are perfectly anti-correlated. If one is heads, the other is tails, every tim...


## Evaluating Trajectories with Additional Histories

**Important:** RULER does not currently support trajectories with `additional_histories`. If you try to use RULER with additional histories, it will raise an error.

Here are two approaches to work around this:

1. **Use a simple reward function** - Evaluate based on the main trajectory only
2. **Create simplified trajectories** - Extract just the main history for RULER evaluation, while keeping the full trajectory with additional histories for training

Let's demonstrate both approaches:

- Approach 1: Simple Reward Function
- Evaluate trajectories based on their main conversation only
- This works well when the additional histories are for training context, not evaluation


In [12]:

def simple_reward_function(trajectory: Trajectory) -> float:
    """
    A simple reward function that evaluates the main trajectory.
    In practice, you'd use more sophisticated logic based on your task.
    """
    # Extract the final assistant message from the main conversation
    final_choice = None
    for msg in reversed(trajectory.messages_and_choices):
        if isinstance(msg, Choice):
            final_choice = msg
            break

    if not final_choice or not final_choice.message.content:
        return 0.0

    content = final_choice.message.content.lower()
    score = 0.0

    # Simple heuristics (replace with your actual evaluation logic)
    # For code tasks: check if solution mentions fixes
    if "fixed" in content or "fix" in content:
        score += 0.3
    if "null" in content or "bug" in content:
        score += 0.2

    # For Q&A tasks: check if answer is substantive
    if len(content) > 100:
        score += 0.3
    if "quantum" in content or "entanglement" in content:
        score += 0.2

    # Bonus for using sub-agents (if metrics indicate it)
    if trajectory.metrics.get("sub_agents_used", 0) > 0:
        score += 0.1

    return min(score, 1.0)

# Evaluate our trajectories
all_trajectories = [
    ("Preserve Tokens", trajectory_preserve_tokens),
    ("Sub-Agents", trajectory_sub_agents),
    ("History Compaction", trajectory_compaction),
]

print("=" * 80)
print("SIMPLE REWARD FUNCTION EVALUATION")
print("=" * 80)

for name, traj in all_trajectories:
    reward = simple_reward_function(traj)
    print(f"\n{name}:")
    print(f"  Computed reward: {reward:.3f}")
    print(f"  Original reward: {traj.reward}")
    print(f"  Additional histories: {len(traj.additional_histories)}")
    print(f"  Metrics: {traj.metrics}")

SIMPLE REWARD FUNCTION EVALUATION

Preserve Tokens:
  Computed reward: 0.000
  Original reward: 0.9
  Additional histories: 1
  Metrics: {'preserved_reasoning_tokens': True}

Sub-Agents:
  Computed reward: 0.900
  Original reward: 0.85
  Additional histories: 2
  Metrics: {'sub_agents_used': 2, 'bugs_found': 3, 'bugs_fixed': 1, 'task_completed': True}

History Compaction:
  Computed reward: 0.500
  Original reward: 0.92
  Additional histories: 1
  Metrics: {'history_compacted': True, 'original_turns': 4, 'compacted_turns': 3}
